In [1]:
import pandas as pd

TEST = '/kaggle/input/house-prices-advanced-regression-techniques/test.csv'
TRAIN = '/kaggle/input/house-prices-advanced-regression-techniques/train.csv'

test_df = pd.read_csv(filepath_or_buffer=TEST)
df = pd.read_csv(filepath_or_buffer=TRAIN,).drop(columns=['Id'])
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [2]:
TARGET = 'SalePrice'
keys = [key for key, value in df.drop(columns=[TARGET]).dtypes.to_dict().items() if str(value) in {'int64', 'float64'}]
COLUMNS = [key for key in keys if test_df[key].isna().sum() == 0]
knn_columns  = [key for key in keys if 0 < test_df[key].isna().sum() < 5]

The competition uses RMSE, so we need an RMSE metric.

In [3]:
import numpy as np
from sklearn.metrics import mean_squared_error

def rmse(y_true, y_pred) -> float:
    # if we have failures due to NaNs we want to localize them
    left = np.log(1 + y_true)
    right = np.log(1 + y_pred)
    return mean_squared_error(squared=False, y_true=left, y_pred=right)


In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df[COLUMNS], df[TARGET], test_size=0.20, random_state=2024)
print('train/test sizes: {}/{}'.format(len(X_train), len(X_test)))

train/test sizes: 1168/292


In [5]:
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split

MODELS = {
    'ridge +': Ridge(tol=1e-4, random_state=2024, max_iter=10000, positive=True, solver='lbfgs'),
}


columns = COLUMNS + ['BsmtFinSF1',  'BsmtUnfSF', 'TotalBsmtSF']
imputer = KNNImputer()
knn_df = imputer.fit_transform(X=df[columns])
for name, model in MODELS.items():
    scores = []
    for random_state in range(20):
        X_train, X_test, y_train, y_test = train_test_split(knn_df, df[TARGET], test_size=0.20, random_state=random_state)
        try:
            model.fit(X=X_train, y=y_train)
            score = rmse(y_true=y_test, y_pred=model.predict(X=X_test))
            scores.append(score)
        except ValueError:
            pass

    print('{:5.4f}'.format(sum(scores)/len(scores),))

0.2085


/tmp/ipykernel_18/3099204580.py:7: RuntimeWarning: invalid value encountered in log
  right = np.log(1 + y_pred)


Baseline (no imputed values) score is 0.2298.

In [6]:
SUBMISSION = '/kaggle/working/submission.csv'

imputer = KNNImputer().fit(X=pd.concat(objs=[df, test_df])[columns])
y_tree_pred = MODELS[name].fit(X=imputer.transform(X=df[columns]), y=df[TARGET]).predict(X=imputer.transform(X=test_df[columns]))
pd.DataFrame(data={'Id': test_df['Id'], 'SalePrice': y_tree_pred}).to_csv(path_or_buf=SUBMISSION, index=False)